In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor
import torchvision.models as models
from PIL import Image
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sb

from functools import partial
from collections import OrderedDict

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_recall_fscore_support

import time

hidden_width = 32
hidden_nblocks = 4
train_max_epoch = 15

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Resnet

In [2]:
class Conv2dAuto(nn.Conv2d):
    # same as Conv2d but with padding
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs) # super() : inherit from baseclass
        self.padding =  (self.kernel_size[0] // 2, self.kernel_size[1] // 2) # dynamic add padding based on the kernel_size
        
conv3x3 = partial(Conv2dAuto, kernel_size=3, bias=False)

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.in_channels, self.out_channels =  in_channels, out_channels
        self.blocks = nn.Identity()
        self.shortcut = nn.Identity()   
    
    def forward(self, x):
        residual = x
        if self.should_apply_shortcut:
            residual = self.shortcut(x)
        # print(residual.shape)
        x = self.blocks(x)
        # print(x.shape)
        x += residual # shapes of x and residual don't match when expansion > 1 and self.blocks = nn.Identity()
        return x
    
    @property # use method like attribute with "." syntax
    def should_apply_shortcut(self):
        return self.in_channels != self.out_channels
    
class ResNetResidualBlock(ResidualBlock):
    def __init__(self, in_channels, out_channels, expansion=1, downsampling=1, conv=conv3x3, *args, **kwargs):
        super().__init__(in_channels, out_channels)
        self.expansion, self.downsampling, self.conv = expansion, downsampling, conv
        self.shortcut = nn.Sequential(OrderedDict(
        {
            # Conv2d expects the input to be of shape [batch_size, input_channels, input_height, input_width]
            'conv' : nn.Conv2d(self.in_channels, self.expanded_channels, kernel_size=1,
                      stride=self.downsampling, bias=False),
            
            'bn' : nn.BatchNorm2d(self.expanded_channels)
        })) if self.should_apply_shortcut else None
        
        
    @property
    def expanded_channels(self):
        return self.out_channels * self.expansion
    
    @property
    def should_apply_shortcut(self):
        return self.in_channels != self.expanded_channels

def conv_bn(in_channels, out_channels, conv, *args, **kwargs):
    return nn.Sequential(OrderedDict({'conv': conv(in_channels, out_channels, *args, **kwargs), 
                                      'bn': nn.BatchNorm2d(out_channels)
                                     }))

class ResNetBasicBlock(ResNetResidualBlock):
    expansion = 1
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, *args, **kwargs)
        self.blocks = nn.Sequential(
            # self.conv = conv3x3
            conv_bn(self.in_channels, self.out_channels, conv=self.conv, bias=False, stride=self.downsampling),
            activation(),
            conv_bn(self.out_channels, self.expanded_channels, conv=self.conv, bias=False),
        )

class ResNetBottleNeckBlock(ResNetResidualBlock):
    expansion = 4
    def __init__(self, in_channels, out_channels, activation=nn.ReLU, *args, **kwargs):
        super().__init__(in_channels, out_channels, expansion=4, *args, **kwargs)
        self.blocks = nn.Sequential(
           conv_bn(self.in_channels, self.out_channels, self.conv, film=self.film, kernel_size=1),
             activation(),
             conv_bn(self.out_channels, self.out_channels, self.conv, film=self.film, kernel_size=3, stride=self.downsampling),
             activation(),
             conv_bn(self.out_channels, self.expanded_channels, self.conv, film=self.film, kernel_size=1),
        )
        

class ResNetLayer(nn.Module):
    # def __init__(self, in_channels, out_channels, gamma, beta, block=ResNetBasicBlock, n=1, *args, **kwargs):
    def __init__(self, in_channels, out_channels, block=ResNetBasicBlock, n=1, *args, **kwargs):
        super().__init__()
        # 'We perform downsampling directly by convolutional layers that have a stride of 2.'
        downsampling = 2 if in_channels != out_channels else 1
        
        self.blocks = nn.Sequential(
            block(in_channels , out_channels, *args, **kwargs, downsampling=downsampling),
            *[block(out_channels * block.expansion, 
                    out_channels, downsampling=1, *args, **kwargs) for _ in range(n - 1)]
        )
    
    # def forward(self, x, gamma, beta)
    def forward(self, x): #gamma beta
        x = self.blocks(x)
        # x = FiLM(x, gamma, beta)
        return x

class ResNetEncoder(nn.Module):
    """
    ResNet encoder composed by increasing different layers with increasing features.
    """
    def __init__(self, in_channels=3, blocks_sizes=[64, 128, 256, 512], deepths=[2,2,2,2], 
                 activation=nn.ReLU, block=ResNetBasicBlock, *args,**kwargs):
        super().__init__()
        
        self.blocks_sizes = blocks_sizes
        
        self.gate = nn.Sequential(
            nn.Conv2d(in_channels, self.blocks_sizes[0], kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(self.blocks_sizes[0]),
            activation(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.in_out_block_sizes = list(zip(blocks_sizes, blocks_sizes[1:]))
        self.blocks = nn.ModuleList([ 
            ResNetLayer(blocks_sizes[0], blocks_sizes[0], n=deepths[0], activation=activation, 
                        block=block,  *args, **kwargs),
            *[ResNetLayer(in_channels * block.expansion, 
                          out_channels, n=n, activation=activation, 
                          block=block, *args, **kwargs) 
              for (in_channels, out_channels), n in zip(self.in_out_block_sizes, deepths[1:])]       
        ])
        
        # gammas[:, num_block, block_dim]
        
        
    def forward(self, x):
        x = self.gate(x)
        for block in self.blocks:
            x = block(x)
        return x

class ResnetDecoder(nn.Module):
    """
    This class represents the tail of ResNet. It performs a global pooling and maps the output to the
    correct class by using a fully connected layer.
    """
    def __init__(self, in_features, n_classes):
        super().__init__()
        self.avg = nn.AdaptiveAvgPool2d((1, 1))
        self.decoder = nn.Linear(in_features, n_classes)

    def forward(self, x):
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.decoder(x)
        return x

class ResNet(nn.Module):
    
    def __init__(self, in_channels, n_classes, *args, **kwargs):
        super().__init__()
        self.encoder = ResNetEncoder(in_channels, *args, **kwargs)
        self.decoder = ResnetDecoder(self.encoder.blocks[-1].blocks[-1].expanded_channels, n_classes)
        # self.module_dim = 
        
    def forward(self, x):
        
        x = self.encoder(x)
        x = self.decoder(x)
        return x

def resnet18(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[2, 2, 2, 2])

def resnet34(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBasicBlock, deepths=[3, 4, 6, 3])

def resnet50(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 6, 3])

def resnet101(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 4, 23, 3])

def resnet152(in_channels, n_classes):
    return ResNet(in_channels, n_classes, block=ResNetBottleNeckBlock, deepths=[3, 8, 36, 3])

## MLP

In [3]:
class mlp(nn.Module):
        def __init__(self, input_size, output_size = 1, hidden_width = 20, hidden_nblocks = 2):
            super(mlp, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            
            self.hidden_width = hidden_width
            self.hidden_nblocks = hidden_nblocks
            
            self.fc1 = nn.Linear(self.input_size, self.hidden_width)
            self.fc2 = nn.Linear(self.hidden_width,self.hidden_width)
            self.fc3 = nn.Linear(self.hidden_width, self.output_size)
            
            self.relu = torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid() ## sigmoid for multi-label, softmax for multi-class (mutually exclusive)
            
        def forward(self, x, film_params):
            out = self.fc1(x)
            out = self.relu(out)
            
            
            for i in range(self.hidden_nblocks):
                out = self.fc2(out)
                
                # ------- film layer -----------
                start = i * hidden_width * 2
                mid = start + hidden_width
                end = mid + hidden_width
                
                gamma = film_params[:, start : mid]
                beta = film_params[:, mid : end]
                
#                 print(out.shape)
#                 print(gamma.shape)
#                 print(beta.shape)
                
                out = out * gamma
                out += beta
                # ------- film layer -----------
                
                out = self.relu(out)
            
            out = self.fc3(out)
            out = self.sigmoid(out)
            return out

## Dataset

In [4]:
def normalize(values):
    # zero mean, unit variance
    return (values-values.mean())/values.std()

def normalize_maxmin(values):
    # range from 0 to 1
    (values-values.min())/(values.max()-values.min())

def preprocess_df(df):
    # convert timecodes to year and month columns
    datetimes = pd.to_datetime(df['time'])
    df['month'] = datetimes.dt.month
    df['year'] = datetimes.dt.year

    df['month_cyclic'] = 7 - abs(df['month'] - 7)

    df['lat_norm'] = normalize(df['latitude'])
    df['lng_norm'] = normalize(df['longitude'])
    df['depth_norm'] = normalize(df['depth'])
    df['year_norm'] = normalize(df['year'])
    df['month_cyclic_norm'] = normalize(df['month_cyclic'])

df = pd.read_csv('data_stephen_fix_header.csv', header=[0])
preprocess_df(df)
    
print(df.shape[0])
print(df['borehole'].nunique())
df.head()

2837
566


,latitude,longitude,time,borehole,depth,frozen,cryostructures,visible_ice,ASTM_2488,materials,...,top_of_interval,bottom_of_interval,month,year,month_cyclic,lat_norm,lng_norm,depth_norm,year_norm,month_cyclic_norm
0,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,0.15,0,NaN,NaN,TOPSOIL,Organics,...,0.0,0.3,3,2012,3,1.439692,1.851129,-1.024010,-1.164786,-1.225079
1,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,0.85,1,NaN,Pure ice,ICE,Ice,...,0.3,1.4,3,2012,3,1.439692,1.851129,-0.835753,-1.164786,-1.225079
2,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,1.90,1,Nf,No visible ice,SW-SM,Coarse till,...,1.4,2.4,3,2012,3,1.439692,1.851129,-0.553369,-1.164786,-1.225079
3,69.16162,-133.08682,2012-03-21T00:00:00Z,0170-1-10,5.40,1,Nf,No visible ice,GW-GM,Coarse till,...,2.4,8.4,3,2012,3,1.439692,1.851129,0.387914,-1.164786,-1.225079
4,69.16105,-133.08880,2012-03-21T00:00:00Z,0170-1-12,1.20,1,Nf,No visible ice,GP-GM,Coarse till,...,0.0,2.4,3,2012,3,1.437542,1.842296,-0.741625,-1.164786,-1.225079


In [5]:
class Geo90Dataset(Dataset):
    def __init__(self, data_root, df, base_lat, base_lng, chip_size=32, label_name = 'frozen'):
        
        self.base_lat = base_lat
        self.base_lng = base_lng
        
        self.df = df
        
        self.chip_size = chip_size
        self.label_name = label_name
        
        self.trans = transforms.ToTensor()
        
        self.preloaded = torch.zeros(26, 6000, 6000)
        
        for i, file in enumerate(os.listdir("geomorph_data")):
            # name = file.split('_')[0]
            # print(name)
            self.preloaded[i] = self.trans(Image.open("geomorph_data/" + file))
        
        print('Dataset initialized')
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        bh_id = row.at['borehole']
        lat = row.at['latitude']
        lng = row.at['longitude']
        

        pixel_len = 5/6000
        

        lat_index_start = np.round((self.base_lat - lat) / pixel_len - self.chip_size/2).astype(int)
        lat_index_end = lat_index_start + self.chip_size
        
        lng_index_start = np.round((lng - self.base_lng) / pixel_len - self.chip_size/2).astype(int)
        lng_index_end = lng_index_start + self.chip_size
        
        image = self.preloaded[:, lat_index_start:lat_index_end,lng_index_start:lng_index_end]
        
        
        # surface = torch.tensor(row.filter(['depth'])).float()
        surface = torch.tensor(row.filter(['depth_norm', 'month_cyclic_norm', 'lat_norm', 'lng_norm', 'year_norm'])).float()
        
        frozen = torch.tensor(row.at['frozen']).float()
        
        return {'image': image, 'surface_data': surface, 'frozen': frozen}

In [6]:
# base_lat = 70
# base_lng = -135

# full_dataset = Geo90Dataset("geomorph_data", df, base_lat, base_lng)

# def prepare_dataloader(full_dataset, batchsize=20):
#     batchsize = 20
#     train_size = int(0.8 * len(full_dataset))
#     test_size = len(full_dataset) - train_size

#     training_data, test_data = torch.utils.data.random_split(full_dataset, [train_size, test_size])

#     trainloader = DataLoader(training_data, batch_size=batchsize, shuffle=True)
#     testloader = DataLoader(test_data, batch_size=batchsize, shuffle=True)
    
#     return trainloader, testloader

## Train model

## FiLM

In [7]:
generator = models.resnet18()

def train_model(trainloader, testloader, print_epochs = False, loss_fn = torch.nn.BCELoss()):
    # loss: binary cross entropy
    n_film_params = hidden_width * hidden_nblocks * 2
    
    # gen_model= resnet18(26, n_film_params)
    gen_model = generator
    gen_model.fc = nn.Linear(512, n_film_params)
    gen_model.conv1 = nn.Conv2d(26, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    # print(gen_model)

    input_size = list(full_dataset[0]['surface_data'].size())
    net_model = mlp(input_size[0],1, hidden_width, hidden_nblocks).to(device)
    
    gen_optimizer = torch.optim.Adam(gen_model.parameters())
    net_optimizer = torch.optim.Adam(net_model.parameters())
    
    gen_model.to(device)
    net_model.to(device)

    # --------- check back propagation ----------- -
    # net_model.fc1.weight.register_hook(lambda x: print('grad accumulated in mlp fc1'))
    # gen_first_layer = gen_model.encoder.blocks[0].blocks[0].blocks[0].conv
    # gen_first_layer.weight.register_hook(lambda x: print('grad accumulated in resnet first layer'))

    epoch_loss = np.zeros([train_max_epoch, 2])
    for epoch in range(train_max_epoch):  # loop over the dataset multiple times

        # ------------ train -----------------
        gen_model.train()
        net_model.train()
        running_loss_sum = 0.0
        for i, data in enumerate(trainloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            images, surface_data, labels = data['image'].to(device), data['surface_data'].to(device), data['frozen'].to(device)

            # TODO: exammine film_params gradients / readup pytorch
            gen_params = gen_model(images)
            predicted = net_model(surface_data, gen_params)
            predicted = torch.squeeze(predicted)
            loss = loss_fn(predicted, labels)

            gen_optimizer.zero_grad()
            net_optimizer.zero_grad()

            loss.backward()

            gen_optimizer.step()
            net_optimizer.step()

            running_loss_sum += loss.item()

        # ----------- get validation loss for current epoch --------------
        gen_model.eval()
        net_model.eval()
        validation_loss_sum = 0.0
        for i, data in enumerate(testloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            images, surface_data, labels = data['image'].to(device), data['surface_data'].to(device), data['frozen'].to(device)

            # TODO: exammine film_params gradients / readup pytorch
            gen_params = gen_model(images)
            predicted = net_model(surface_data, gen_params)
            predicted = torch.squeeze(predicted)
            loss = loss_fn(predicted, labels)
            validation_loss_sum += loss.item()

        # ---------------- print statistics ------------------------

        running_loss = running_loss_sum / len(trainloader)
        validation_loss = validation_loss_sum / len(testloader)
        epoch_loss[epoch, :] =  [running_loss, validation_loss]
        
        if print_epochs:
            print('epoch %2d: running loss: %.5f, validation loss: %.5f' %
                          (epoch + 1, running_loss, validation_loss))

        torch.save(gen_model.state_dict(), os.path.join('mlp-resnet-models/', 'gen-epoch-{}.pt'.format(epoch+1)))
        torch.save(net_model.state_dict(), os.path.join('mlp-resnet-models/', 'net-epoch-{}.pt'.format(epoch+1)))

    if print_epochs:
        print('Finished Training')
    
    return epoch_loss

## Test model

def test_model(epoch_loss, print_model_epoch = False):
    
    # ------ select model ---------
    ind = np.argmin(epoch_loss[:, 1])
    
    n_film_params = hidden_width * hidden_nblocks * 2
    
    gen_model = generator
    gen_model.fc = nn.Linear(512, n_film_params)
    gen_model.conv1 = nn.Conv2d(26, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)

    input_size = list(full_dataset[0]['surface_data'].size())
    net_model = mlp(input_size[0],1, hidden_width, hidden_nblocks)

    gen_model.load_state_dict(torch.load('mlp-resnet-models/gen-epoch-{}.pt'.format(ind+1)))
    net_model.load_state_dict(torch.load('mlp-resnet-models/net-epoch-{}.pt'.format(ind+1)))
    
    gen_model.to(device)
    net_model.to(device)
    
    if print_model_epoch:
        print("epoch {} model selected".format(ind+1))
    
    # evaluate model on test set
    gen_model.eval()
    net_model.eval()
    with torch.no_grad():
        y_test = []
        y_pred = []
        for i, data in enumerate(testloader, 0):
            images, surface_data, labels = data['image'].to(device), data['surface_data'].to(device), data['frozen'].to(device)

            # y_test.append(label.numpy().list())
            # print(label.shape)
            # print(images.shape)

            gen_params = gen_model(images)
            predicted = net_model(surface_data, gen_params)
            predicted = torch.squeeze(predicted)

            predicted = torch.round(predicted)
            # print(predicted.shape)
            lb = labels.tolist()
            pr = predicted.tolist()
            y_test.extend(lb)
            y_pred.extend(pr)
    
    arr_accuracy = accuracy_score(y_test, y_pred)
    scores = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=0)
    return arr_accuracy, scores


#     print(confusion_matrix(y_test,y_pred))
#     print(classification_report(y_test,y_pred))
#     print(accuracy_score(y_test, y_pred))

## Pure MLP

## MLP

In [8]:
class mlp_pure(nn.Module):
        def __init__(self, input_size, output_size):
            super(mlp_pure, self).__init__()
            self.input_size = input_size
            self.output_size = output_size
            
            self.hidden_size = hidden_width
            self.hidden_nblocks = hidden_nblocks
            
            self.fc1 = torch.nn.Linear(self.input_size, self.hidden_size)
            self.fc2 = torch.nn.Linear(self.hidden_size,self.hidden_size)
            self.fc3 = torch.nn.Linear(self.hidden_size, self.output_size)
            
            self.relu = torch.nn.ReLU()
            self.sigmoid = torch.nn.Sigmoid() ## sigmoid for multi-label, softmax for multi-class (mutually exclusive)
            
        def forward(self, x):
            out = self.fc1(x)
            out = self.relu(out)
            # print(out.shape)
            
            for i in range(self.hidden_nblocks):
                out = self.fc2(out)
                out = self.relu(out)
            
            out = self.fc3(out)
            out = self.sigmoid(out)
            return out

def train_mlp(trainloader, testloader, print_epochs = False, loss_fn = torch.nn.BCELoss()):
    input_size = list(full_dataset[0]['surface_data'].size())
    surface_model = mlp_pure(input_size[0],1)
    
    surface_model.to(device)
    
    optimizer = torch.optim.Adam(surface_model.parameters())

    epoch_loss = np.zeros([train_max_epoch, 2])
    for epoch in range(train_max_epoch):  # loop over the dataset multiple times

        surface_model.train()
        running_loss_sum = 0.0
        for i, data in enumerate(trainloader, 0): # loop over each sample

            # get the inputs; data is a list of [inputs, labels]
            surface_data, labels = data['surface_data'].to(device), data['frozen'].to(device)

            predicted = surface_model(surface_data)

            loss = loss_fn(predicted.squeeze(), labels)

            optimizer.zero_grad()

            loss.backward()

            optimizer.step()

            running_loss_sum += loss.item()

        # ----------- get validation loss for current epoch --------------
        surface_model.eval()
        validation_loss_sum = 0.0
        for i, data in enumerate(testloader, 0): # loop over each sample

            surface_data, labels = data['surface_data'].to(device), data['frozen'].to(device)

            predicted = surface_model(surface_data)

            loss = loss_fn(predicted.squeeze(), labels)

            validation_loss_sum += loss.item()

        # ---------------- print statistics ------------------------

        running_loss = running_loss_sum / len(trainloader)
        validation_loss = validation_loss_sum / len(testloader)
        epoch_loss[epoch, :] =  [running_loss, validation_loss]
        
        if print_epochs:
            print('epoch %2d: running loss: %.5f, validation loss: %.5f' %
                          (epoch + 1, running_loss, validation_loss))

        torch.save(surface_model.state_dict(), os.path.join('mlp-models/', 'epoch-{}.pt'.format(epoch+1)))
    
    if print_epochs:
        print('Finished Training')
        
    return epoch_loss
        
def test_mlp(epoch_loss, print_model_epoch = False):
    
    # ------ select model ---------
    ind = np.argmin(epoch_loss[:, 1])
    
    input_size = list(full_dataset[0]['surface_data'].size())
    
    surface_model = mlp_pure(input_size[0],1)
    surface_model.load_state_dict(torch.load('mlp-models/epoch-{}.pt'.format(ind+1)))
    
    surface_model.to(device)
    
    if print_model_epoch:
        print("epoch {} model selected".format(ind+1))
    
    # evaluate model on test set
    surface_model.eval()

    with torch.no_grad():
        y_test = []
        y_pred = []
        for i, data in enumerate(testloader, 0):
            surface_data, labels = data['surface_data'].to(device), data['frozen'].to(device)

            # y_test.append(label.numpy().list())
            # print(label.shape)
            # print(images.shape)

            output = surface_model(surface_data)
            predicted = torch.round(output)
            # print(predicted.shape)
            lb = labels.tolist()
            pr = predicted.tolist()
            y_test.extend(lb)
            y_pred.extend(pr)
    
    arr_accuracy = accuracy_score(y_test, y_pred)
    scores = precision_recall_fscore_support(y_test, y_pred, average=None, zero_division=0)
    return arr_accuracy, scores


## Multiple Runs

In [9]:
base_lat = 70
base_lng = -135

full_dataset = Geo90Dataset("geomorph_data", df, base_lat, base_lng, chip_size = 256)

Dataset initialized


In [10]:
batchsize = 20
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

training_data, test_data = torch.utils.data.random_split(full_dataset, [train_size, test_size])

trainloader = DataLoader(training_data, batch_size=batchsize, shuffle=True)
testloader = DataLoader(test_data, batch_size=batchsize, shuffle=True)

In [ ]:
max_iterations = 15
results = np.zeros([max_iterations, 9])
# trainloader, testloader = prepare_dataloader(full_dataset)

for it in range(max_iterations):
    start = time.time()
    
    # mlp
#     epoch_loss_mlp = train_mlp(trainloader,testloader, print_epochs = True)
#     acc, scores = test_mlp(epoch_loss_mlp, print_model_epoch = True)
    
    #     # ------- mlp-resnet film 
    epoch_loss = train_model(trainloader, testloader, print_epochs=True)
    acc, scores = test_model(epoch_loss, print_model_epoch = True)
    
    results[it, 0:2] = scores[0]
    results[it, 2:4] = scores[1]
    results[it, 4:6] = scores[2]
    results[it, 6:8] = scores[3]
    results[it, 8] = acc 
    
    end = time.time()
    
    print('iteration {} elapsed time: {}, accuracy : {}'.format(it+1, end-start, acc))


c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


epoch  1: running loss: 0.47121, validation loss: 0.39187
epoch  2: running loss: 0.32995, validation loss: 0.25656
epoch  3: running loss: 0.32311, validation loss: 0.37639
epoch  4: running loss: 0.36195, validation loss: 0.37176
epoch  5: running loss: 0.25852, validation loss: 0.19292
epoch  6: running loss: 0.37093, validation loss: 0.22609


In [ ]:
def display_table(scores):
    df = np.reshape(scores, [2,4], order ='F')
    df = pd.DataFrame(df)
    
    # df.style.set_table_attributes("style='display:inline'").set_caption(mode)
    
    df.columns = ['precision', 'recall', 'f1', 'support']
    df.index = ['unfrozen', 'frozen']
    
    display(df)
def display_results(results):
    mean = np.mean(results, axis=0)
    std = np.std(results, axis=0)
    
    print("mean")
    display_table(mean[0:8])
    
    print("std")
    display_table(std[0:8])
    
    print("Accuracy mean: {}, std: {}".format(mean[8], std[8]))

display_results(results)